<a href="https://colab.research.google.com/github/Norberto27/Norberto27/blob/main/NaiveBayes_Ver1_Norberto_Martearena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Version 1**
Al final del notebook, figuran los avances, Detalles y Experiencia de ambas versiones

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## **Seleccionar 5 documentos al azar**

In [ ]:
import random
# Seleccionamos 5 documentos al azar del conjunto de entrenamiento
import random

# Fijar una semilla aleatoria
random.seed(27)

# Seleccionar 5 párrafos al azar
random_docs = random.sample(newsgroups_train.data, 5)


# **Medir la similaridad de esos 5 documentos con el resto**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcular la similaridad de los documentos seleccionados con todos los documentos del conjunto
for i, doc in enumerate(random_docs):
    doc_vect = tfidfvect.transform([doc])  # Vectoriza el documento individual
    cossim = cosine_similarity(doc_vect, X_train)[0]  # Similaridad con todos los documentos de train
    mostsim = np.argsort(cossim)[::-1][1:6]  # 5 documentos más similares, excluyendo el mismo documento

    # Muestra la clase del documento seleccionado
    print(f'Documento {i} pertenece a la clase: {newsgroups_train.target_names[y_train[newsgroups_train.data.index(doc)]]}')

    # Muestra los 5 documentos más similares
    print(f'Los 5 documentos más similares son:')
    for idx in mostsim:
        print(f'Clase: {newsgroups_train.target_names[y_train[idx]]}')



Documento 0 pertenece a la clase: soc.religion.christian
Los 5 documentos más similares son:
Clase: alt.atheism
Clase: talk.religion.misc
Clase: soc.religion.christian
Clase: soc.religion.christian
Clase: soc.religion.christian
Documento 1 pertenece a la clase: rec.autos
Los 5 documentos más similares son:
Clase: sci.electronics
Clase: sci.electronics
Clase: comp.os.ms-windows.misc
Clase: soc.religion.christian
Clase: rec.autos
Documento 2 pertenece a la clase: rec.sport.hockey
Los 5 documentos más similares son:
Clase: rec.sport.hockey
Clase: rec.sport.baseball
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Documento 3 pertenece a la clase: rec.motorcycles
Los 5 documentos más similares son:
Clase: rec.motorcycles
Clase: misc.forsale
Clase: misc.forsale
Clase: misc.forsale
Clase: misc.forsale
Documento 4 pertenece a la clase: comp.os.ms-windows.misc
Los 5 documentos más similares son:
Clase: comp.graphics
Clase: sci.crypt
Clase: comp.os.ms-windows.misc
Clase: 

# **Entrenar modelos Naïve Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# Entrenamos el modelo Naive Bayes Multinomial
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Vectorizamos los datos de prueba y predecimos
X_test = tfidfvect.transform(newsgroups_test.data)
y_pred = clf.predict(X_test)

# Calculamos el F1-score con promediado macro
print(f'F1-score (macro): {f1_score(newsgroups_test.target, y_pred, average="macro")}')


F1-score (macro): 0.5854345727938506


# **Transponer la matriz y estudiar la similaridad entre palabras**

In [ ]:
# Transponemos la matriz documento-término
X_train_transposed = X_train.T

# Elegimos manualmente 5 palabras para estudiar su similaridad
palabras = ['car', 'computer', 'government', 'money', 'health']

for palabra in palabras:
    idx = tfidfvect.vocabulary_[palabra]
    palabra_vect = X_train_transposed[idx]  # Vectoriza la palabra individualmente
    similar_words = cosine_similarity(palabra_vect, X_train_transposed)[0]  # Similaridad con otras palabras
    most_similar_idx = np.argsort(similar_words)[::-1][1:6]  # Las 5 palabras más similares
    print(f'Palabra: {palabra}')
    for idx in most_similar_idx:
        print(f'Palabra similar: {idx2word[idx]}')


Palabra: car
Palabra similar: cars
Palabra similar: criterium
Palabra similar: civic
Palabra similar: owner
Palabra similar: dealer
Palabra: computer
Palabra similar: decwriter
Palabra similar: harkens
Palabra similar: deluged
Palabra similar: shopper
Palabra similar: the
Palabra: government
Palabra similar: the
Palabra similar: to
Palabra similar: of
Palabra similar: libertarian
Palabra similar: encryption
Palabra: money
Palabra similar: buyback
Palabra similar: computerize
Palabra similar: fundraising
Palabra similar: to
Palabra similar: spend
Palabra: health
Palabra similar: ohip
Palabra similar: provincial
Palabra similar: care
Palabra similar: traditionalists
Palabra similar: fiscally


# **Informe de Resultados: Comparación de Modelos Naive Bayes**
1. Introducción
El objetivo de este análisis fue entrenar y evaluar modelos de clasificación utilizando Naive Bayes, con el fin de maximizar el F1-score (macro) en un conjunto de datos de texto (datasets grupos de noticias). Para esto, se exploraron dos enfoques diferentes de vectorización: uno básico (Versión 1) y otro con ajustes de hiperparámetros (Versión 2). A través de esta comparación, se buscó entender cómo la modificación en los parámetros del TfidfVectorizer afecta el rendimiento del modelo.
2. Descripción de las versiones
Durante el análisis, se probaron dos enfoques distintos para la vectorización de los datos:
Enfoque 1: Vectorización básica (Versión 1)
En la primera versión, se utilizó el método básico de vectorización de texto utilizando TfidfVectorizer con sus valores predeterminados. Este enfoque permite transformar los textos en vectores sin realizar ningún tipo de ajuste o filtro adicional.
•	TfidfVectorizer(): Usado con los parámetros por defecto.
Enfoque 2: Ajuste de hiperparámetros (Versión 2)
En la segunda versión, se realizaron ajustes en los hiperparámetros del TfidfVectorizer para mejorar la representación de los textos y ver si esto impactaba en el rendimiento del modelo.
•	TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2), sublinear_tf=True):
o	max_df=0.8: Excluye términos muy frecuentes que aparecen en más del 80% de los documentos.
o	min_df=2: Excluye términos que aparecen en menos de 2 documentos.
o	ngram_range=(1, 2): Considera tanto unigramas como bigramas.
o	sublinear_tf=True: Aplica una transformación sublineal a las frecuencias de término.
3. Resultados
Se midió el F1-score (macro) para ambas versiones con el fin de evaluar el rendimiento de cada enfoque. A continuación, se presenta una comparación de los resultados obtenidos:
F1-score de la versión 1 (Vectorización básica):
•	F1-score (macro): 0.5854345727938506
F1-score de la versión 2 (Ajuste de hiperparámetros):
•	F1-score (macro): 0.5745979561373709
Comparación de resultados:
El rendimiento del modelo mostró diferencias entre ambos enfoques. En la versión básica, se obtuvo un F1-score inicial que sirvió como línea base. Tras ajustar los hiperparámetros, se observó 0.5746, ligeramente más bajo que el original de 0.5854.
Posibles razones del comportamiento observado:
•	Enfoque 1 (básico): Al no realizar ningún ajuste en los parámetros, el modelo se entrenó con una representación más general de los textos, lo que puede haber permitido capturar términos relevantes, pero también introducir ruido.
•	Enfoque 2 (ajuste de hiperparámetros): Los ajustes en la vectorización permitieron filtrar términos muy frecuentes y poco representativos, además de considerar combinaciones de palabras (bigramas), lo que podría haber mejorado la calidad de la representación de los textos. Sin embargo, en este caso, el ajuste no logró mejorar el F1-score. Esto puede deberse a que los bigramas no aportaron información adicional suficiente o a que el modelo Naive Bayes no se beneficia tanto de este tipo de ajustes en algunos conjuntos de datos.
4. Conclusión final
El proceso de ajuste de hiperparámetros es una etapa fundamental en el desarrollo de modelos de aprendizaje automático. En este análisis, a pesar de los ajustes realizados en la vectorización de los textos, el rendimiento en términos de F1-score (macro) no mejoró en comparación con el enfoque básico.
Sin embargo, el aprendizaje obtenido durante este proceso fue valioso, ya que me permitió entender mejor cómo afectan las diferentes configuraciones de los parámetros a la representación de los datos y al rendimiento del modelo. Además, comprobe que los ajustes de hiperparámetros no siempre garantizan una mejora en el rendimiento, pero forman parte esencial del ciclo iterativo de optimización de modelos.
Este análisis será una base para futuras mejoras, donde podrían probarse otros algoritmos o enfoques para mejorar el rendimiento del modelo de clasificación.


# **Reflexion Final (Experiencia)**

Problemas Enfrentados y Experiencia Adicional
Durante el desarrollo de este proyecto, me surgieron varios desafíos que enriquecieron mi experiencia y permitieron un aprendizaje más profundo sobre la implementación de modelos de clasificación basados en Naive Bayes. Estos problemas abarcaron tanto la preparación de los datos como la optimización de los modelos. A continuación, hago un breve  resumen de los principales problemas y lecciones aprendidas:
1.	Vectorización de Datos Textuales:
o	Problema: La representación de los textos fue uno de los principales retos. Al trabajar con datos textuales, elegir una adecuada vectorización era crucial para obtener buenos resultados. En la primera versión, se utilizó un enfoque básico de TfidfVectorizer, que arrojó resultados decentes, pero no óptimos.
o	Lección: La elección de los parámetros de la vectorización tiene un impacto significativo en la calidad de los datos utilizados por el modelo. Sin embargo, pequeños cambios en los hiperparámetros no siempre producen mejoras considerables, lo que lleva a la importancia de experimentar y probar múltiples configuraciones.
2.	Ajustes de Hiperparámetros:
o	Problema: Al ajustar los hiperparámetros de la vectorización, se esperaba mejorar el rendimiento del modelo. No obstante, en algunos casos los cambios no produjeron una mejora en el F1-score, lo que podría atribuirse a la naturaleza del conjunto de datos y la idoneidad del modelo Naive Bayes para este tipo de tareas.
o	Lección: El ajuste de hiperparámetros no es una solución garantizada para mejorar el rendimiento, y a veces los cambios pueden no ser suficientes o incluso degradar el rendimiento. Esto refuerza la importancia de probar diversas combinaciones de hiperparámetros, así como la necesidad de profundizar en el entendimiento de cómo el modelo interactúa con los datos.
3.	Aleatoriedad y Semillas de Datos:
o	Problema: La segunda ejecución (con ajuste en hiperparametros) los resultados fueron distintos porque por error se tomaron 5 documentos al azar diferentes. Por ello, se fijo una semilla aleatoria.
o	Lección: Fijar semillas aleatorias es fundamental cuando se trabaja con selección de muestras aleatorias. Sin embargo, es importante también entender que ciertos algoritmos o funciones pueden depender de múltiples factores, y la variabilidad en los resultados puede reflejar la naturaleza del conjunto de datos o el impacto del procesamiento de características.
4.	Similaridad entre Documentos:
o	Problema: Al medir la similaridad de los documentos seleccionados, algunos resultados mostraron cierta dispersión entre clases, lo que evidenció la complejidad del conjunto de datos. En varios casos, documentos de clases distintas aparecieron como los más similares, lo que llevó a una revisión del modelo de vectorización.
o	Lección: La similaridad entre documentos no siempre es intuitiva, y un análisis más detallado de los datos podría ser necesario para entender las relaciones que el modelo detecta. Esto invita a explorar otros enfoques como redes neuronales o técnicas más avanzadas de procesamiento de lenguaje natural (NLP).
Reflexión Final
Cada uno de estos desafíos presentó una oportunidad de aprendizaje que permitió mejorar la comprensión tanto de los algoritmos como de los datos con los que se trabajaba. La experiencia adquirida al resolver estos problemas será valiosa en futuros proyectos, tanto para ajustar la vectorización de textos como para aplicar modelos más adecuados y técnicas de ajuste de hiperparámetros más avanzadas.
